# Parameters 

In [1]:
fileName = '4PpKk.hdf5'
dsNameStates = '4PpKk-fullStateBool'
dsNameWDLs = '4PpKk-Wdl'
convertStates = False
nPi = 4
nPa = 2
nWPa = 1
input_shape = (4,8,8)
batch_size = 4096
epochs = 500
dataDiv = 1
num_classes = 5

multiGPU = False
whichGPU = 0

# Imports

In [2]:
# ### To select which GPU to use, import these before importing Keras or Tensorflow
# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# # The GPU id to use, usually either &quot;0&quot; or &quot;1&quot;
# os.environ["CUDA_VISIBLE_DEVICES"]="0"  
 
# # Do other imports now...

In [3]:

if(multiGPU):
    from keras.utils.training_utils import multi_gpu_model
else:
    import os
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
    # The GPU id to use, usually either "0" or "1"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(whichGPU)

# from __future__ import print_function
import keras
# from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
# import pandas as pd
from sklearn.model_selection import train_test_split
import sys
import h5py
import numpy as np

Using TensorFlow backend.


## Analysis
```
3PKk-Wdl-Buffered == 3PKk-Wdl-Seq

4PpKk-Wdl == 4PpKk-Wdl-Buffered != 4PpKk-Wdl-retry

5PPpKk-Wdl != 5PPpKk-Wdl-Buffered
    from 50.000 then buffered != 10, but wdl == 10
```
  
### WDL score count
#### 3PKk-WDL-Seq
[0, 0, 125024, 0, 124960]

# Functions for vector to full state conversion AND MORE!

In [4]:
def sq2hnit(sq):
    col = sq%8
    row = (sq - col)//8
    return col,row

# 0: pawns
# 1: kings
def vecSt2fullSt(vecSt, nPi, nPa, nWPa):
    fullSt = np.zeros((4,8,8), dtype = 'bool')
    for i in range(nPi - 2):
        sq = vecSt[i]
        col,row = sq2hnit(sq)
        if i < nWPa:
            fullSt[0][row][col] = True
        else:
            fullSt[1][row][col] = True
    col,row = sq2hnit(vecSt[-2])
    fullSt[2][row][col] = True
    col,row = sq2hnit(vecSt[-1])
    fullSt[3][row][col] = True
    return fullSt 

def vecSt2fullSt_8x8x2(vecSt, nPi, nPa, nWPa):
    fullSt = np.zeros((8,8,2), dtype = 'int8')
    for i in range(nPi - 2):
        sq = vecSt[i]
        col,row = sq2hnit(sq)
        if i < nWPa:
            fullSt[row][col][0] = 1
        else:
            fullSt[row][col][0] = -1
    col,row = sq2hnit(vecSt[-2])
    fullSt[row][col][1] = 1
    col,row = sq2hnit(vecSt[-1])
    fullSt[row][col][1] = -1
    return fullSt 
# nPi = 3
# nPa = 1
# nWPa = 1
# vecSt2fullSt(d3[0], nPi, nPa, nWPa)
# nPi = 5
# nPa = 3
# nWPa = 2
# vecSt2fullSt(d5[0], nPi, nPa, nWPa)
    

# count nr of each score instance
# wdlCounter placeholders: [-2, -1, 0, 1 ,2]

def wdlCountingMachine(ds):
    wdlCounter = [0,0,0,0,0]
    l = len(ds)
    i = 0
    intv = l//100
    for wdl in ds:
        i += 1
        if i%intv == 0:
            sys.stdout.write(str((i*100)//l) + " percentage")
            sys.stdout.write('\r')
            sys.stdout.flush()
        wdlCounter[wdl[0] + 2] += 1
    print(wdlCounter)
    return wdlCounter
# wdlCountingMachine(d3t)

# Load data

In [5]:
# load datasets
f = h5py.File(fileName, 'r')
d = f[dsNameStates]
dt = f[dsNameWDLs]
# d5 = f['5PPpKk']
l = len(d)

if convertStates:
    X = np.array([vecSt2fullSt(vecSt,nPi, nPa, nWPa) for vecSt in d[:l//dataDiv]])
else:
    X = d[:l//dataDiv]

y = dt[:l//dataDiv]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

del X
del y

print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

X_train shape: (10216846, 4, 8, 8)
y_train shape: (10216846, 1)
X_test shape: (5032178, 4, 8, 8)
y_test shape: (5032178, 1)
10216846 train samples
5032178 test samples


In [6]:
# del f['4PpKk-fullState']
# f = h5py.File(fileName,'a')
# d = f['4PpKk']
# X = np.array([vecSt2fullSt(vecSt,nPi, nPa, nWPa) for vecSt in d[:l//dataDiv]])
# f.create_dataset('4PpKk-fullState', data = X)
# for i in f:print(i)
# f.close()


# Create Keras model

In [22]:
# import keras.backend as K
# K.set_floatx('float16')
# K.set_epsilon(1e-4) #default is 1e-7
# K.set_floatx('float32')
# K.set_epsilon(1e-7) #default is 1e-7

model = Sequential()
filterDivider = 1
l1filters = 8//filterDivider
l2filters = 16//filterDivider
l3filters = 32//filterDivider
print(l1filters)
print(l2filters)
print(l3filters)

model.add(Conv2D(l1filters, kernel_size=(3, 3),
                 padding='valid',
                 activation='relu',
                 data_format = "channels_first",
#                  kernel_initializer = 
                 input_shape=input_shape))
model.add(Conv2D(l2filters, kernel_size=(3, 3),
                 padding='valid',
                 activation='relu'))
model.add(Conv2D(l3filters, kernel_size=(3, 3),
                 padding='valid',
                 activation='relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
if multiGPU: 
    model = keras.utils.multi_gpu_model(model, gpus=2)
model.summary()
model.save_weights('tempWeights.hdf5')


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


# model.save_weights('tempWeights.hdf5')
# model.load_weights('tempWeights.hdf5')

8
16
32
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 8, 6, 6)           296       
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 6, 4, 16)          880       
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 4, 2, 32)          4640      
_________________________________________________________________
flatten_11 (Flatten)         (None, 256)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 1285      
Total params: 7,101
Trainable params: 7,101
Non-trainable params: 0
_________________________________________________________________


# Train

In [23]:
model.fit(X_train, y_train,
      batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 10216846 samples, validate on 5032178 samples
Epoch 1/500
10216846/10216846 [==============================] - 45s 4us/step - loss: 0.4130 - acc: 0.8205 - val_loss: 0.3095 - val_acc: 0.8629
Epoch 2/500
10216846/10216846 [==============================] - 44s 4us/step - loss: 0.2682 - acc: 0.8841 - val_loss: 0.2392 - val_acc: 0.8970
Epoch 3/500
10216846/10216846 [==============================] - 45s 4us/step - loss: 0.2258 - acc: 0.9036 - val_loss: 0.2205 - val_acc: 0.9055
Epoch 4/500
10216846/10216846 [==============================] - 44s 4us/step - loss: 0.2076 - acc: 0.9119 - val_loss: 0.2101 - val_acc: 0.9088
Epoch 5/500
10216846/10216846 [==============================] - 44s 4us/step - loss: 0.1975 - acc: 0.9165 - val_loss: 0.1899 - val_acc: 0.9197
Epoch 6/500
10216846/10216846 [==============================] - 44s 4us/step - loss: 0.1901 - acc: 0.9197 - val_loss: 0.1903 - val_acc: 0.9184
Epoch 7/500
10216846/10216846 [==============================] - 44s 4us/step - l

10216846/10216846 [==============================] - 45s 4us/step - loss: 0.1364 - acc: 0.9431 - val_loss: 0.1412 - val_acc: 0.9422
Epoch 58/500
10216846/10216846 [==============================] - 44s 4us/step - loss: 0.1362 - acc: 0.9432 - val_loss: 0.1376 - val_acc: 0.9423
Epoch 59/500
10216846/10216846 [==============================] - 44s 4us/step - loss: 0.1360 - acc: 0.9432 - val_loss: 0.1384 - val_acc: 0.9423
Epoch 60/500
10216846/10216846 [==============================] - 45s 4us/step - loss: 0.1358 - acc: 0.9433 - val_loss: 0.1418 - val_acc: 0.9396
Epoch 61/500
10216846/10216846 [==============================] - 45s 4us/step - loss: 0.1354 - acc: 0.9435 - val_loss: 0.1429 - val_acc: 0.9407
Epoch 62/500
10216846/10216846 [==============================] - 45s 4us/step - loss: 0.1352 - acc: 0.9435 - val_loss: 0.1396 - val_acc: 0.9416
Epoch 63/500
10216846/10216846 [==============================] - 45s 4us/step - loss: 0.1350 - acc: 0.9437 - val_loss: 0.1352 - val_acc: 0.943

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



10216846/10216846 [==============================] - 45s 4us/step - loss: 0.1259 - acc: 0.9476 - val_loss: 0.1323 - val_acc: 0.9453
Epoch 129/500
10216846/10216846 [==============================] - 45s 4us/step - loss: 0.1258 - acc: 0.9477 - val_loss: 0.1315 - val_acc: 0.9449
Epoch 130/500
10216846/10216846 [==============================] - 45s 4us/step - loss: 0.1257 - acc: 0.9477 - val_loss: 0.1280 - val_acc: 0.9473
Epoch 131/500
10216846/10216846 [==============================] - 45s 4us/step - loss: 0.1255 - acc: 0.9477 - val_loss: 0.1381 - val_acc: 0.9430
Epoch 132/500
10216846/10216846 [==============================] - 45s 4us/step - loss: 0.1256 - acc: 0.9477 - val_loss: 0.1278 - val_acc: 0.9472
Epoch 133/500
 3805184/10216846 [==========>...................] - ETA: 22s - loss: 0.1255 - acc: 0.9478

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

10216846/10216846 [==============================] - 45s 4us/step - loss: 0.1152 - acc: 0.9521 - val_loss: 0.1235 - val_acc: 0.9487
Epoch 476/500
10216846/10216846 [==============================] - 45s 4us/step - loss: 0.1152 - acc: 0.9520 - val_loss: 0.1195 - val_acc: 0.9503
Epoch 477/500
10216846/10216846 [==============================] - 45s 4us/step - loss: 0.1151 - acc: 0.9521 - val_loss: 0.1194 - val_acc: 0.9508
Epoch 478/500
 3051520/10216846 [=======>......................] - ETA: 24s - loss: 0.1150 - acc: 0.9522

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



10216846/10216846 [==============================] - 45s 4us/step - loss: 0.1150 - acc: 0.9521 - val_loss: 0.1162 - val_acc: 0.9521
Epoch 494/500
10216846/10216846 [==============================] - 45s 4us/step - loss: 0.1150 - acc: 0.9521 - val_loss: 0.1162 - val_acc: 0.9516
Epoch 495/500
 4743168/10216846 [============>.................] - ETA: 18s - loss: 0.1149 - acc: 0.9522

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



10216846/10216846 [==============================] - 45s 4us/step - loss: 0.1149 - acc: 0.9522 - val_loss: 0.1258 - val_acc: 0.9476
Epoch 499/500
10216846/10216846 [==============================] - 45s 4us/step - loss: 0.1149 - acc: 0.9521 - val_loss: 0.1169 - val_acc: 0.9504
Epoch 500/500
10216846/10216846 [==============================] - 45s 4us/step - loss: 0.1148 - acc: 0.9522 - val_loss: 0.1214 - val_acc: 0.9494


In [24]:
# for i in range(7):
#     batch_size = 64 * 2 ** i
#     print("batch_size: ", batch_size)
#     model.fit(X_train, y_train,
#           batch_size=batch_size,
#               epochs=1,
#               verbose=1,
#               validation_data=(X_test, y_test))

In [25]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

KeyboardInterrupt: 

In [ ]:
model.save_weights('4PpKk_3_weights.hdf5')